# Implementação RBF para dataset

In [190]:
#Determinar a média e Desvio padrão das funções Radiais
# como o conjunto tem 3 classes, pode-se considerar um centro de cada grupo 
# (centroide dos conjuntos de atributos dos elementos de cada grupo) e 
# escolher uma função de base radial para tentar ajustar os dados.

def f_mu_sigma(df,lista):
    x_train_, x_test_, y_train_, y_test_ = train_test_split(df.drop(lista, axis = 1), 
                                                        df[lista], 
                                                        test_size = 0.3, 
                                                        random_state = random.seed(123))

    XC = pd.DataFrame(x_train_)

    del x_train_ , x_test_ , y_train_ , y_test_ 

    mu      = np.array(XC.groupby('type').agg(np.average))
    sigma   = np.array(XC.groupby('type').agg(np.std))
    
    return mu, sigma

In [191]:
# Funções Radiais

def radial(X, mu, sigma,tipo):
    
    if tipo == 'gaussian':
        return np.sum(np.exp(- (X-mu)**2 / (2 * sigma**2)),axis=1)
    
    elif tipo == 'multiquadrica':
        return np.sum(np.power((X-mu)**2 + sigma**2, 0.5),axis=1)

    
  

In [192]:
# Função de ativação Linear (Adaline)

def f_net(activation):
    return np.where(activation >= 0, 1, 0)
    #return activation 



In [193]:
# Etapa Foward

def f_Forward(X,w_output_layer,net_out, fnet_out):
    # Camada de saída
    net_out = np.sum(X*w_output_layer,axis=1)
    fnet_out =  f_net(net_out)
    
    return net_out, fnet_out



In [194]:
# RBF

def rbf_delta(phi,Y,n_inputs,n_hidden,n_outputs ,eta ,epoch):

    #Vetor após clusterização com bias
    Xp = np.concatenate((phi.T, np.reshape(np.ones(len(X)),(len(X),1))), axis=1)
    
    # Inicialização dos pesos de forma aleatória
    w_output_layer = np.array([[random.uniform(-0.5,0.5) for i in range(n_hidden+1)] for i in range(n_outputs)])

    ## Output Layer
    net_out  = np.zeros(n_outputs)
    fnet_out = np.zeros(n_outputs)
  
    for e in range(epoch):
        squaredError = 0
        for i in range(len(Y)):
            # Vetor de entrada com acréscimo de coluna para bias
            net_out, fnet_out = f_Forward(Xp[i],w_output_layer,net_out,fnet_out)

            #Cálculo do Erro
            error = Y[i] - fnet_out
            squaredError = squaredError + np.sum(error**2)

            #Treinamento
            w_output_layer = w_output_layer + np.dot(np.reshape(eta*error,(n_hidden,1)),np.reshape(Xp[i],(1,n_hidden+1)))
              
        
        squaredError = squaredError / len(Y)
        print(squaredError)

    
    return  squaredError , w_output_layer, fnet_out

In [195]:
# Predição de novos dados

def f_predict(X,w_output_layer):

    return  np.argmax(f_net(np.dot(X,w_output_layer)),axis=1)


In [196]:
import numpy as np
from math import exp
import random
import pandas as pd
#tratamentos de dataframes
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


random.seed(123)

# Importação do conjunto de dados Wine sem cabeçalho
df = pd.read_csv("/home/jovyan/MLPxRBF/wine.csv",delimiter=',',header=None)

# Nomeação das colunas
dict_colnames = {
0: 'type',
1: 'alcohol',
2: 'malic_acid',
3: 'ash',
4: 'alcalinity_ash',
5: 'magnesium',
6: 'total_phenols',
7: 'flavanoids',
8: 'nonflavanoid_phenols',
9: 'proanthocyanins',
10: 'color_intensity',
11: 'hue',
12: 'OD280_OD315_diluted_wines',
13: 'proline'
   }
    
df = df.rename(columns= dict_colnames)

# Criação do vetor de base canônica
df['p1'] = np.where(df['type']==1, 1, 0)
df['p2'] = np.where(df['type']==2, 1, 0)
df['p3'] = np.where(df['type']==3, 1, 0)

# Objetos auxiliares para split
lista = ['p1','p2','p3']
lista2 = ['type','p1','p2','p3']

# Primeiro estágio: Não supervisionado
mu, sigma = f_mu_sigma(df,lista)


# Separação entre dados de treino e teste
x_train, x_test, y_train, y_test = train_test_split(df.drop(lista2, axis = 1), 
                                                    df[lista], 
                                                    test_size = 0.3, 
                                                    random_state = random.seed(123))

# Matriz para treinar a rede
X   = np.array(x_train.copy())
Y   = np.array(y_train.copy())

## Dimensões da Rede
n_inputs = len(X[0])
n_outputs = len(Y[0])
n_hidden = 3

# Cálculo de phi
phi = np.zeros((n_hidden,len(X)))

for i in range(n_hidden):
    phi[i] = radial(X, mu[i], sigma[i],tipo = 'gaussian')


Erro , w_output_layer, fnet_out = rbf_delta(phi,Y,n_inputs,n_hidden,n_outputs ,eta = 0.1,epoch = 100000)


0.5
0.28225806451612906
0.25
0.23387096774193547
0.1774193548387097
0.21774193548387097
0.20967741935483872
0.18548387096774194
0.20967741935483872
0.16129032258064516
0.20161290322580644
0.1935483870967742
0.1774193548387097
0.1935483870967742
0.1935483870967742
0.20967741935483872
0.18548387096774194
0.1693548387096774
0.20967741935483872
0.14516129032258066
0.21774193548387097
0.18548387096774194
0.18548387096774194
0.14516129032258066
0.20967741935483872
0.1532258064516129
0.21774193548387097
0.1532258064516129
0.1935483870967742
0.20967741935483872
0.13709677419354838
0.20967741935483872
0.18548387096774194
0.13709677419354838
0.18548387096774194
0.18548387096774194
0.1532258064516129
0.18548387096774194
0.12096774193548387
0.1774193548387097
0.16129032258064516
0.20161290322580644
0.1532258064516129
0.20161290322580644
0.13709677419354838
0.16129032258064516
0.0967741935483871
0.13709677419354838
0.1693548387096774
0.13709677419354838
0.1693548387096774
0.1693548387096774
0.18548

0.12096774193548387
0.12903225806451613
0.11290322580645161
0.14516129032258066
0.12903225806451613
0.12903225806451613
0.1693548387096774
0.11290322580645161
0.11290322580645161
0.0967741935483871
0.12096774193548387
0.10483870967741936
0.12903225806451613
0.12096774193548387
0.0967741935483871
0.12903225806451613
0.12903225806451613
0.0967741935483871
0.10483870967741936
0.0967741935483871
0.0967741935483871
0.1532258064516129
0.1935483870967742
0.13709677419354838
0.08870967741935484
0.1532258064516129
0.08870967741935484
0.13709677419354838
0.12903225806451613
0.0967741935483871
0.10483870967741936
0.11290322580645161
0.12903225806451613
0.18548387096774194
0.13709677419354838
0.08870967741935484
0.1532258064516129
0.11290322580645161
0.08064516129032258
0.11290322580645161
0.12903225806451613
0.1774193548387097
0.11290322580645161
0.12903225806451613
0.12903225806451613
0.10483870967741936
0.18548387096774194
0.12903225806451613
0.08870967741935484
0.13709677419354838
0.1209677419

0.0967741935483871
0.08064516129032258
0.11290322580645161
0.14516129032258066
0.13709677419354838
0.1532258064516129
0.08064516129032258
0.08064516129032258
0.0967741935483871
0.11290322580645161
0.13709677419354838
0.16129032258064516
0.10483870967741936
0.11290322580645161
0.13709677419354838
0.1693548387096774
0.13709677419354838
0.12096774193548387
0.12903225806451613
0.11290322580645161
0.08870967741935484
0.08064516129032258
0.0967741935483871
0.08064516129032258
0.0967741935483871
0.08870967741935484
0.11290322580645161
0.12903225806451613
0.08064516129032258
0.0967741935483871
0.06451612903225806
0.0967741935483871
0.12903225806451613
0.14516129032258066
0.1774193548387097
0.12096774193548387
0.10483870967741936
0.11290322580645161
0.13709677419354838
0.11290322580645161
0.12903225806451613
0.1532258064516129
0.10483870967741936
0.12096774193548387
0.16129032258064516
0.12096774193548387
0.11290322580645161
0.1532258064516129
0.1532258064516129
0.11290322580645161
0.1129032258

0.0967741935483871
0.11290322580645161
0.11290322580645161
0.11290322580645161
0.11290322580645161
0.11290322580645161
0.11290322580645161
0.12903225806451613
0.11290322580645161
0.0967741935483871
0.0967741935483871
0.08064516129032258
0.0967741935483871
0.10483870967741936
0.11290322580645161
0.11290322580645161
0.08064516129032258
0.11290322580645161
0.11290322580645161
0.12903225806451613
0.1693548387096774
0.11290322580645161
0.12903225806451613
0.12903225806451613
0.11290322580645161
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.08064516129032258
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.10483870967741936
0.11290322580645161
0.11290322580645161
0.08064516129032258
0.13709677419354838
0.12096774193548387
0.11290322580645161
0.08064516129032258
0.08064516129032258
0.08064516129032258
0.0967741935483871
0.12903225806451613
0.12903225806451613
0.12096774193548387
0.14516129032258066
0.1209677419354

0.11290322580645161
0.08064516129032258
0.11290322580645161
0.11290322580645161
0.11290322580645161
0.0967741935483871
0.10483870967741936
0.08064516129032258
0.0967741935483871
0.08064516129032258
0.06451612903225806
0.0967741935483871
0.0967741935483871
0.08064516129032258
0.0967741935483871
0.11290322580645161
0.13709677419354838
0.14516129032258066
0.12096774193548387
0.0967741935483871
0.11290322580645161
0.11290322580645161
0.0967741935483871
0.10483870967741936
0.11290322580645161
0.0967741935483871
0.14516129032258066
0.12096774193548387
0.08064516129032258
0.12096774193548387
0.14516129032258066
0.12096774193548387
0.0967741935483871
0.0967741935483871
0.08064516129032258
0.0967741935483871
0.11290322580645161
0.0967741935483871
0.12903225806451613
0.11290322580645161
0.11290322580645161
0.14516129032258066
0.1532258064516129
0.16129032258064516
0.11290322580645161
0.12096774193548387
0.12903225806451613
0.10483870967741936
0.08064516129032258
0.08064516129032258
0.08870967741

...

0.12096774193548387
0.14516129032258066
0.13709677419354838
0.11290322580645161
0.08064516129032258
0.08064516129032258
0.0967741935483871
0.0967741935483871
0.11290322580645161
0.10483870967741936
0.13709677419354838
0.13709677419354838
0.12903225806451613
0.0967741935483871
0.08870967741935484
0.08870967741935484
0.10483870967741936
0.12096774193548387
0.12903225806451613
0.10483870967741936
0.08870967741935484
0.08870967741935484
0.12903225806451613
0.16129032258064516
0.10483870967741936
0.0967741935483871
0.0967741935483871
0.08870967741935484
0.08064516129032258
0.0967741935483871
0.11290322580645161
0.11290322580645161
0.1532258064516129
0.14516129032258066
0.12096774193548387
0.12903225806451613
0.11290322580645161
0.10483870967741936
0.10483870967741936
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.08870967741935484
0.08870967741935484
0.11290322580645161
0.0967741935483871
0.08064516129032258
0.11290322580645161
0.11290322580645161
0.0967741935

0.0967741935483871
0.0967741935483871
0.056451612903225805
0.11290322580645161
0.1532258064516129
0.11290322580645161
0.0967741935483871
0.08064516129032258
0.08064516129032258
0.08064516129032258
0.10483870967741936
0.08870967741935484
0.13709677419354838
0.14516129032258066
0.12096774193548387
0.1693548387096774
0.12096774193548387
0.12096774193548387
0.0967741935483871
0.12096774193548387
0.12096774193548387
0.12903225806451613
0.11290322580645161
0.07258064516129033
0.07258064516129033
0.0967741935483871
0.13709677419354838
0.12903225806451613
0.13709677419354838
0.10483870967741936
0.08870967741935484
0.08870967741935484
0.08870967741935484
0.11290322580645161
0.10483870967741936
0.10483870967741936
0.0967741935483871
0.08064516129032258
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.10483870967741936
0.08064516129032258
0.12903225806451613
0.14516129032258066
0.12903225806451613
0.08870967741935484
0.08870967741935484
0.08870967741935484
0.10483870967741936
0.11290322

0.16129032258064516
0.12096774193548387
0.11290322580645161
0.12096774193548387
0.16129032258064516
0.0967741935483871
0.10483870967741936
0.12903225806451613
0.11290322580645161
0.12903225806451613
0.12903225806451613
0.12903225806451613
0.0967741935483871
0.08064516129032258
0.08064516129032258
0.0967741935483871
0.08870967741935484
0.08064516129032258
0.08064516129032258
0.08870967741935484
0.10483870967741936
0.0967741935483871
0.12096774193548387
0.10483870967741936
0.12903225806451613
0.12096774193548387
0.12903225806451613
0.08870967741935484
0.10483870967741936
0.12903225806451613
0.10483870967741936
0.10483870967741936
0.08870967741935484
0.10483870967741936
0.06451612903225806
0.08064516129032258
0.0967741935483871
0.0967741935483871
0.10483870967741936
0.08870967741935484
0.12903225806451613
0.16129032258064516
0.11290322580645161
0.10483870967741936
0.10483870967741936
0.10483870967741936
0.0967741935483871
0.07258064516129033
0.10483870967741936
0.12096774193548387
0.07258

0.0967741935483871
0.08064516129032258
0.0967741935483871
0.11290322580645161
0.0967741935483871
0.13709677419354838
0.11290322580645161
0.12903225806451613
0.12096774193548387
0.13709677419354838
0.12903225806451613
0.12903225806451613
0.10483870967741936
0.11290322580645161
0.12096774193548387
0.12903225806451613
0.11290322580645161
0.12903225806451613
0.12903225806451613
0.10483870967741936
0.10483870967741936
0.12903225806451613
0.12903225806451613
0.08870967741935484
0.08064516129032258
0.08064516129032258
0.07258064516129033
0.08870967741935484
0.0967741935483871
0.0967741935483871
0.0967741935483871
0.08064516129032258
0.11290322580645161
0.08870967741935484
0.11290322580645161
0.0967741935483871
0.12096774193548387
0.1532258064516129
0.13709677419354838
0.11290322580645161
0.08870967741935484
0.08870967741935484
0.0967741935483871
0.10483870967741936
0.10483870967741936
0.12903225806451613
0.08870967741935484
0.10483870967741936
0.10483870967741936
0.10483870967741936
0.1048387

0.0967741935483871
0.08064516129032258
0.0967741935483871
0.10483870967741936
0.11290322580645161
0.08064516129032258
0.11290322580645161
0.12096774193548387
0.0967741935483871
0.07258064516129033
0.07258064516129033
0.11290322580645161
0.07258064516129033
0.10483870967741936
0.11290322580645161
0.07258064516129033
0.12096774193548387
0.10483870967741936
0.12903225806451613
0.16129032258064516
0.11290322580645161
0.10483870967741936
0.0967741935483871
0.11290322580645161
0.0967741935483871
0.0967741935483871
0.08064516129032258
0.10483870967741936
0.13709677419354838
0.0967741935483871
0.08870967741935484
0.13709677419354838
0.1693548387096774
0.11290322580645161
0.08870967741935484
0.08870967741935484
0.0967741935483871
0.12096774193548387
0.12096774193548387
0.11290322580645161
0.0967741935483871
0.08064516129032258
0.0967741935483871
0.11290322580645161
0.10483870967741936
0.11290322580645161
0.0967741935483871
0.08064516129032258
0.06451612903225806
0.08870967741935484
0.1451612903

## Predito

In [197]:
# Matriz para teste da rede
X_teste   = np.array(x_test.copy())

In [198]:
phi_teste = np.zeros((n_hidden,len(X_teste)))

In [199]:
for i in range(n_hidden):
    phi_teste[i] = radial(X_teste, mu[i], sigma[i],tipo = 'gaussian')

In [200]:
Xp = np.concatenate((phi_teste.T, np.reshape(np.ones(len(X_teste)),(len(X_teste),1))), axis=1)

In [201]:
f_predict(Xp,w_output_layer.T)

array([1, 1, 1, 1, 0, 0, 1, 2, 0, 2, 2, 1, 2, 2, 0, 1, 2, 2, 1, 1, 1, 1,
       0, 1, 2, 1, 2, 1, 1, 2, 1, 2, 0, 1, 2, 0, 1, 2, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 2, 0, 0, 2, 2, 2, 0])

## Real

In [202]:
np.argmax(np.array(y_test),axis=1)

array([1, 1, 1, 1, 0, 0, 1, 2, 0, 2, 2, 1, 2, 2, 0, 1, 2, 2, 1, 1, 1, 1,
       0, 1, 2, 1, 2, 0, 1, 2, 1, 2, 0, 1, 2, 0, 1, 2, 1, 0, 1, 1, 1, 0,
       2, 0, 0, 2, 0, 0, 2, 2, 2, 1])

# RBF

## Matriz de Confusão

In [203]:
from sklearn.metrics import confusion_matrix
confusion_matrix(np.argmax(np.array(y_test),axis=1), f_predict(Xp,w_output_layer.T))

array([[13,  1,  0],
       [ 1, 21,  0],
       [ 1,  0, 17]])

## Acurácia

In [204]:
from sklearn.metrics import accuracy_score
accuracy_score(np.argmax(np.array(y_test),axis=1), f_predict(Xp,w_output_layer.T))

0.9444444444444444